## Custom Sampling / Initial Population Creation


Directly converted from R code which uses an 1D array formulation for 8 weeks, so the gene is (1728, )

- (4+2)*8*36
- where there is 4 products category
- 2 decision variable; display, feature
- time horizon of 8 weeks
- 36 products per category

In [4]:
import numpy as np
from pymoo.core.sampling import Sampling

GAobj = {
    'popSize': 10,
    'nBits': 1728,
    'nc': 36,
    'h': 8,
    'price_ga': 4,
    'ndf': 2,
    'Lim_pr_low': 23.04,
    'Lim_d': 0,
    'Lim_f': 21.96,
    'Lim_dr_t': 0,
    'Lim_fr_t': 4.56,
}

class SKUPopulationSampling(Sampling):

    def __init__(self, GAobj):
        super().__init__()
        self.GAobj = GAobj

    def _do(self, problem, n_samples, **kwargs):
        # Extracting parameters from GAobj for clarity
        GAobj = self.GAobj
        pop_size = GAobj['popSize']
        pop_nBits = GAobj['nBits']
        nc = GAobj['nc']
        h = GAobj['h']
        price_ga = GAobj['price_ga']
        ndf = GAobj['ndf']
        Lim_pr_low = GAobj['Lim_pr_low']
        Lim_d = GAobj['Lim_d']
        Lim_f = GAobj['Lim_f']
        Lim_dr_t = GAobj['Lim_dr_t']
        Lim_fr_t = GAobj['Lim_fr_t']
        
        # Initializing the population matrix
        sku_pop = np.zeros((pop_size, pop_nBits), dtype=int)
        
        for s in range(pop_size):
            x_sku_pop = np.zeros((nc, (price_ga + ndf) * h), dtype=int)
            
            d_nt = np.zeros(nc, dtype=int)
            f_nt = np.zeros(nc, dtype=int)
            p_nt = np.zeros(nc, dtype=int)
            
            for pop_t in range(h):
                t_index_start = pop_t * 6
                t_index_end = (pop_t + 1) * 6 - 1
                
                rnum_pr = np.random.choice(np.arange(1, nc), int(np.ceil(Lim_pr_low)), replace=False)
                p_nt[rnum_pr] += 1
                
                rnum_d = np.zeros(nc, dtype=int)
                sample_d = np.random.choice(np.arange(nc), int(np.floor(Lim_d)), replace=False)
                rnum_d[sample_d] = 1
                d_nt[rnum_d == 1] += 1
                
                rnum_f = np.zeros(nc, dtype=int)
                sample_f = np.random.choice(np.arange(nc), int(np.floor(Lim_f)), replace=False)
                rnum_f[sample_f] = 1
                f_nt[rnum_f == 1] += 1
                
                # Handling promotions
                if len(rnum_pr) > 0:
                    for pr_i in rnum_pr:
                        # Random multinomial distribution equivalent in Python
                        x_sku_pop[pr_i, t_index_start:t_index_start + 3] = np.random.multinomial(1, [0.2, 0.2, 0.2], size=1).T.flatten()
                        
                x_sku_pop[:, t_index_start + 4] = rnum_d
                x_sku_pop[:, t_index_start + 5] = rnum_f
            
            # Flatten and assign to the population matrix
            sku_pop[s, :] = x_sku_pop.flatten()
        
        return sku_pop

In [10]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np

class PromotionOptimizationProblem(ElementwiseProblem):
    def __init__(self):
        self.product_num = 4
        self.discount_num = 1
        self.feature_num = 1
        self.constraint_num = 0
        self.week_count = 8
        self.category_num = 36

        #should be this? but ref_impl code it this way
        # n_var = (self.product_num * (self.discount_num + self.feature_num)) * self.week_count * self.category_num
        n_var = (self.product_num + self.discount_num + self.feature_num) * self.week_count * self.category_num

        self.lim_pr_low = 23.04
        self.lim_d = 0
        self.lim_f=21.96
        self.dr_t=0
        self.fr_t=4.56

        super().__init__(n_var=n_var, n_obj=1, n_constr=self.constraint_num, xl=np.zeros(n_var), xu=np.ones(n_var))



    def _evaluate(self, x, out, *args, **kwargs):
        profit = 0
        # todo

        out["F"] = -profit
        # out["G"] = constraints

problem = PromotionOptimizationProblem()

algorithm = GA(pop_size=10, sampling=SKUPopulationSampling(GAobj=GAobj))

res = minimize(problem,
               algorithm,
               #todo use custom sampling as mentioned
               ('n_gen', 1),
               seed=1,
               verbose=True)



n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       10 |  0.000000E+00 |  0.000000E+00


In [11]:
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [0 0 0 ... 0 0 1]
F = [0.]


In [12]:
res.X.shape

(1728,)

In [14]:
output_file_path = "output.txt"
with open(output_file_path, "w") as file:
    for item in res.X:
        file.write(str(item) + "\n")

print("Output saved to", output_file_path)

Output saved to output.txt


In [42]:
import numpy as np
from pymoo.core.sampling import Sampling

GAobj = {
    'popSize': 1,
    'nBits': 180,
    'nc': 30, # reduce to 1 
    'h': 1, #reduce to 1 week
    'price_ga': 4,
    'ndf': 2,
    'Lim_pr_low': 23.04,
    'Lim_d': 0,
    'Lim_f': 21.96,
    'Lim_dr_t': 0,
    'Lim_fr_t': 4.56,
}

class SKUPopulationSampling(Sampling):

    def __init__(self, GAobj):
        super().__init__()
        self.GAobj = GAobj

    def _do(self, problem=None, n_samples=None, **kwargs):
        # Extracting parameters from GAobj for clarity
        GAobj = self.GAobj
        pop_size = GAobj['popSize']
        pop_nBits = GAobj['nBits']
        nc = GAobj['nc']
        h = GAobj['h']
        price_ga = GAobj['price_ga']
        ndf = GAobj['ndf']
        Lim_pr_low = GAobj['Lim_pr_low']
        Lim_d = GAobj['Lim_d']
        Lim_f = GAobj['Lim_f']
        Lim_dr_t = GAobj['Lim_dr_t']
        Lim_fr_t = GAobj['Lim_fr_t']
        
        # Initializing the population matrix
        sku_pop = np.zeros((pop_size, pop_nBits), dtype=int)
        
        for s in range(pop_size):
            x_sku_pop = np.zeros((nc, (price_ga + ndf) * h), dtype=int)
            
            d_nt = np.zeros(nc, dtype=int)
            f_nt = np.zeros(nc, dtype=int)
            p_nt = np.zeros(nc, dtype=int)
            
            for pop_t in range(h):
                t_index_start = pop_t * 6
                t_index_end = (pop_t + 1) * 6 - 1
                
                rnum_pr = np.random.choice(np.arange(1, nc), int(np.ceil(Lim_pr_low)), replace=False)
                p_nt[rnum_pr] += 1
                num_ones = np.count_nonzero(p_nt == 1)
                print(f"{num_ones= }, {p_nt=}")
                
                rnum_d = np.zeros(nc, dtype=int)
                sample_d = np.random.choice(np.arange(nc), int(np.floor(Lim_d)), replace=False)
                rnum_d[sample_d] = 1
                d_nt[rnum_d == 1] += 1
                num_ones_1 = np.count_nonzero(d_nt == 1)
                print(f"{num_ones_1=}, {d_nt=}")
                
                rnum_f = np.zeros(nc, dtype=int)
                sample_f = np.random.choice(np.arange(nc), int(np.floor(Lim_f)), replace=False)
                rnum_f[sample_f] = 1
                f_nt[rnum_f == 1] += 1
                num_ones_2 = np.count_nonzero(f_nt == 1)
                print(f"{num_ones_2=}, {f_nt=}")
                
                # Handling promotions
                if len(rnum_pr) > 0:
                    for pr_i in rnum_pr:
                        # Random multinomial distribution equivalent in Python
                        x_sku_pop[pr_i, t_index_start:t_index_start + 3] = np.random.multinomial(1, [0.2, 0.2, 0.2], size=1).T.flatten()
                        
                x_sku_pop[:, t_index_start + 4] = rnum_d
                x_sku_pop[:, t_index_start + 5] = rnum_f
            
            # Flatten and assign to the population matrix
            sku_pop[s, :] = x_sku_pop.flatten()
        
        return sku_pop
    
test = SKUPopulationSampling(GAobj)._do()



num_ones= 24, p_nt=array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1])
num_ones_1=0, d_nt=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])
num_ones_2=21, f_nt=array([1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 1])


In [43]:
test

array([[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1]])

In [33]:
test.shape

(1, 180)

In [44]:
np.count_nonzero(test == 1)

45

The number of 1s from the initial pop matches the constraints designed.